Hunter Bennett | Glass Lab | Brain Aging Project | 19 Feb 2021

This notebook finds peaks and annotates them with H3K27Ac reads for downstream analysis. The steps accomplished are as follows:
1. Call both variable width and nucleosome free regions using HOMER.
2. Merge peaks into timepoint specific merged peak sets and overall merged peak sets.
3. Annotate overall merged peak sets with H3K27Ac reads from all tag directories.

In [16]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
import sys
%matplotlib inline
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 200
sns.set(font_scale=1)
sns.set_context('talk')
sns.set_style('white')

# import custom functions
import sys
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from plotting_scripts import label_point, pca_rpkm_mat
from homer_preprocessing import read_annotated_peaks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Set working paths

In [17]:
dataDirectory = '/data/mm10/Brain_MPSIIIA/ChIP/H3K27AC/PU_1/WT/'
inputDirectory = '/data/mm10/Brain_MPSIIIA/ChIP/input/PU1/'
workingDirectory = '/home/h1bennet/brain_aging/results/00_PU1_H3K27Ac/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)

# Call peaks using matched inputs.

## Set list of tag directories

In [18]:
tagdirs = ['03_mouse_BL6_M_9week_PU1_ChIP_H3K27ac_1_JOS_20190809_CTTGTA',
           '05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_1_AL_20200925_AGGTTATA_CAGTTCCG',
           '05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_2_AL_20200925_TCTGTTGG_TCGAATGG',
           '05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_3_AL_20201111_CTGCTTCC_GATAGATC',
           '06_mouse_MPSIIIAhet_M_P240_PU1_ChIP_H3K27ac_1_AL_20191226_ATTCCT',
           '06_mouse_MPSIIIAhet_M_P240_PU1_ChIP_H3K27ac_2D_JOS_20191122_CTTGTA',
           '06_mouse_MPSIIIAhet_M_P240_PU1_ChIP_H3K27ac_2_AL_20191122_CTTGTA',
           '07_mouse_BL6_M_23month_PU1_ChIP_H3K27ac_1_AL_20201121_ATGTAAGT_ACTCTATG',
           '07_mouse_BL6_M_25month_PU1_ChIP_H3K27ac_1_JOS_20191018_ACTTGA',
           '07_mouse_BL6_M_25month_PU1_ChIP_H3K27ac_2_JOS_20191018_AGTTCC',
           '07_mouse_BL6_M_28month_PU1_ChIP_H3K27ac_1_AL_20201121_AACGTTCC_GGAGTACT',
           '07_mouse_BL6_M_31month_PU1_ChIP_H3K27ac_1_AL_20201111_GAACCGCG_TGACCTTA']

## Set list of corresponding input directories

In [19]:
inputdirs = ['01_mouse_BL6_M_8week_PU1_input_3_AL_20191226_CATGGC',
           '02_mouse_MPSIIIAhet_M_4month_PU1_ChIP_input_AL_l20200925_ATCCACTG_AGGTGCGT',
           '02_mouse_MPSIIIAhet_M_4month_PU1_ChIP_input_AL_l20200925_ATCCACTG_AGGTGCGT',
           '02_mouse_MPSIIIAhet_M_4month_PU1_ChIP_input_AL_l20200925_ATCCACTG_AGGTGCGT',
           'mouse_C57_M_PU1_ChIP_input_MPSIIIAWTP240PU1_408_AL_l20191122_CTAGCT',
           'mouse_C57_M_PU1_ChIP_input_MPSIIIAWTP240PU1_408_AL_l20191122_CTAGCT',
           'mouse_C57_M_PU1_ChIP_input_MPSIIIAWTP240PU1_408_AL_l20191122_CTAGCT',
           '04_mouse_BL6_M_26month_PU1_ChIP_month_AL_l20200911_TCATCCTT_AGCGAGCT',
           '04_mouse_BL6_M_26month_PU1_ChIP_month_AL_l20200911_TCATCCTT_AGCGAGCT',
           '04_mouse_BL6_M_26month_PU1_ChIP_month_AL_l20200911_TCATCCTT_AGCGAGCT',
           '04_mouse_BL6_M_26month_PU1_ChIP_month_AL_l20200911_TCATCCTT_AGCGAGCT',
           '04_mouse_BL6_M_26month_PU1_ChIP_month_AL_l20200911_TCATCCTT_AGCGAGCT']

# Call Peaks with matched inputs

In [22]:
if not os.path.isdir('./peak_files'):
    os.mkdir('./peak_files')
    
with open('./peakCalling_homer.sh', 'w') as f:
    for tagdir, inputdir in zip(tagdirs, inputdirs):
        print('analyzing:', tagdir)
        print('input:', inputdir)
        print()
        
        find_peaks_vw = ['findPeaks', dataDirectory + '/' + tagdir,
                          '-i', inputDirectory + '/' + inputdir,
                          '-region',
                          '-size 1000 -minDist 2500',
                          '-o',
                          workingDirectory + '/peak_files/' + tagdir + '_variablewidth_peaks.tsv',
                          '&', '\n\n']
        
        find_peaks_nfr = ['findPeaks', dataDirectory + '/' + tagdir,
                          '-i', inputDirectory + '/' + inputdir,
                          '-nfr', '-size 200',
                          '-o',
                          workingDirectory + '/peak_files/' + tagdir + '_nfr_peaks.tsv',
                          '&', '\n\n']

        # write commands to file
        f.write(' '.join(find_peaks_vw))
        f.write(' '.join(find_peaks_nfr))      
        
    f.close()

analyzing: 03_mouse_BL6_M_9week_PU1_ChIP_H3K27ac_1_JOS_20190809_CTTGTA
input: 01_mouse_BL6_M_8week_PU1_input_3_AL_20191226_CATGGC

analyzing: 05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_1_AL_20200925_AGGTTATA_CAGTTCCG
input: 02_mouse_MPSIIIAhet_M_4month_PU1_ChIP_input_AL_l20200925_ATCCACTG_AGGTGCGT

analyzing: 05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_2_AL_20200925_TCTGTTGG_TCGAATGG
input: 02_mouse_MPSIIIAhet_M_4month_PU1_ChIP_input_AL_l20200925_ATCCACTG_AGGTGCGT

analyzing: 05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_3_AL_20201111_CTGCTTCC_GATAGATC
input: 02_mouse_MPSIIIAhet_M_4month_PU1_ChIP_input_AL_l20200925_ATCCACTG_AGGTGCGT

analyzing: 06_mouse_MPSIIIAhet_M_P240_PU1_ChIP_H3K27ac_1_AL_20191226_ATTCCT
input: mouse_C57_M_PU1_ChIP_input_MPSIIIAWTP240PU1_408_AL_l20191122_CTAGCT

analyzing: 06_mouse_MPSIIIAhet_M_P240_PU1_ChIP_H3K27ac_2D_JOS_20191122_CTTGTA
input: mouse_C57_M_PU1_ChIP_input_MPSIIIAWTP240PU1_408_AL_l20191122_CTAGCT

analyzing: 06_mouse_MPSIIIAhet_M_P240_PU1_ChIP_

# Merge peaks

In [23]:
%%bash
if [ ! -d ./merged_peaks/ ]; then mkdir ./merged_peaks; fi
mergePeaks ./peak_files/*variablewidth* > merged_peaks/vw_peaks_merged.txt
mergePeaks ./peak_files/*nfr_peaks.tsv* > merged_peaks/nfr_peaks_merged.txt

Process is interrupted.


### Make subgroup specific merged peaks - these will be used later

In [24]:
%%bash
if [ ! -d ./merged_peaks/ ]; then mkdir ./merged_peaks; fi
mergePeaks ./peak_files/05_mouse*nfr* > merged_peaks/05_PU1_4month_nfr_peaks_merged.txt
mergePeaks ./peak_files/06_mouse*nfr* > merged_peaks/06_PU1_P240_nfr_peaks_merged.txt
mergePeaks ./peak_files/07_mouse*nfr* > merged_peaks/07_PU1_20MonthPlus_nfr_peaks_merged.txt
mergePeaks ./peak_files/05_mouse*variablewidth* > merged_peaks/05_PU1_4month_vw_peaks_merged.txt
mergePeaks ./peak_files/06_mouse*variablewidth* > merged_peaks/06_PU1_P240_vw_peaks_merged.txt
mergePeaks ./peak_files/07_mouse*variablewidth* > merged_peaks/07_PU1_20MonthPlus_vw_peaks_merged.txt

	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_1_AL_20200925_AGGTTATA_CAGTTCCG_nfr_peaks.tsv (26743 total) and ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_1_AL_20200925_AGGTTATA_CAGTTCCG_nfr_peaks.tsv (26743 total)
	Comparing ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_1_AL_20200925_AGGTTATA_CAGTTCCG_nfr_peaks.tsv (26743 total) and ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_2_AL_20200925_TCTGTTGG_TCGAATGG_nfr_peaks.tsv (27370 total)
	Comparing ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_1_AL_20200925_AGGTTATA_CAGTTCCG_nfr_peaks.tsv (26743 total) and ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_3_AL_20201111_CTGCTTCC_GATAGATC_nfr_peaks.tsv (20945 total)
	Comparing ./peak_files/05_mouse_MPSIIIAhet_M_4month_PU1_ChIP_H3K27ac_2_AL_20200925_TCTGTTGG_TCGAATGG_nfr_peaks.tsv (27370 total) and ./peak_files/05_mouse_MPSIIIA

## Convert merged peaks to bed files for upload to UCSC

In [30]:
%%bash
if [ ! -d ./bed_files/ ]; then mkdir ./bed_files; fi

# delete the existing script file
rm ./make_bed.sh
# create a script file
touch ./make_bed.sh

for peakfile in ./merged_peaks/*;
do bedfile=${peakfile/merged_peaks/bed_files};
bedfile=${bedfile/.txt/.bed}
echo "pos2bed.pl $peakfile > $bedfile" >> make_bed.sh
done


Best to add labels to the bed files so that we can use them on the browser

In [26]:
%%bash

echo 'track name="PU1_H3K27Ac_NFR" description="PU1 Nuclei H3K27Ac Chip-seq Nucleosome Free Regions"' \
| cat - ./bed_files/nfr_peaks_merged.bed \
> temp && mv temp ./bed_files/nfr_peaks_merged.bed

echo 'track name="PU1_H3K27Ac_VW" description="PU1 Nuclei H3K27Ac Chip-seq Variable Width Peaks"' \
| cat - ./bed_files/vw_peaks_merged.bed \
> temp && mv temp ./bed_files/vw_peaks_merged.bed

# Annotate peaks

In [27]:
tagdirs_full = [dataDirectory+i for i in tagdirs]

In [28]:
if not os.path.isdir('./annotated_peaks/'):
    os.mkdir('./annotated_peaks/')

with open('./annotatePeaks_homer.sh', 'w') as f:
    
    annotate_nfr_raw = ['annotatePeaks.pl', './merged_peaks/nfr_peaks_merged.txt',
                       'mm10', '-size 1000', '-raw', '-d \\\n',
                       ' \\\n'.join(tagdirs_full), '>',
                       './annotated_peaks/ann_raw_nfr_peaks_merged.txt &\n\n']
    
    annotate_nfr_norm = ['annotatePeaks.pl', './merged_peaks/nfr_peaks_merged.txt',
                       'mm10', '-size 1000', '-norm 1e7', '-d \\\n',
                       ' \\\n'.join(tagdirs_full), '>',
                       './annotated_peaks/ann_norm_nfr_peaks_merged.txt &\n\n']
    
    annotate_vw_raw = ['annotatePeaks.pl', './merged_peaks/vw_peaks_merged.txt',
                   'mm10', '-size given', '-raw', '-d \\\n',
                   ' \\\n'.join(tagdirs_full), '>',
                   './annotated_peaks/ann_raw_vw_peaks_merged.txt &\n\n']

    annotate_vw_norm = ['annotatePeaks.pl', './merged_peaks/vw_peaks_merged.txt',
                       'mm10', '-size given', '-norm 1e7', '-d \\\n',
                       ' \\\n'.join(tagdirs_full), '>',
                       './annotated_peaks/ann_norm_vw_peaks_merged.txt &']

    f.write(' '.join(annotate_nfr_raw))    
    f.write(' '.join(annotate_nfr_norm))
    f.write(' '.join(annotate_vw_raw))    
    f.write(' '.join(annotate_vw_norm))
    
    f.close()